In [ ]:
# keywords

# pandas.DataFrame.aggregate, aggregation_json, модуль requests, модуль lxml, lxml.html, xpath, xml.etree
# xml.etree.ElementTree

# Чтение и запись данных. Часть 1

## 1. Чтение текстовых файлов, файловые дескрипторы

### 1.1

**Простая задача** сформируйте объект класса dict, где ключ - это IP адрес, а значение - количество раз, которые этот ip встретился в тесте.

*Результат выполнения*
<pre>
{'192.168.101.4': 1, '192.168.102.3': 2, '192.168.7.46': 1}
</pre>

In [1]:
file_path = './data/uwsgi.log'
ip_counter = dict()

with open(file_path, 'r', encoding='utf8') as f:
    for row in f:
        parted_row = row.split(' ')[0]
        if parted_row in ip_counter:
            ip_counter[parted_row] += 1
        else:
            ip_counter[parted_row] = 1
print(ip_counter)

{'192.168.101.4': 1, '192.168.102.3': 2, '192.168.7.46': 1}


### 1.2
**Задача среднего уровня** Распечатайте список URL, к которым происходил доступ. GET или POST параметры печатать не нужно.

*Результат выполнения*
<pre>
/movie/is_personalizable/
/logger/content/time/
/movie/collection/items/recommendations/
/movie/recommendations/
</pre>

In [2]:
file_path = './data/uwsgi.log'

with open(file_path, 'r', encoding='utf8') as f:
    for row in f:
        parted_row = row.split(' ')
        url_list = parted_row[6].split('?')[0]
        print(url_list)

/movie/is_personalizable/
/logger/content/time/
/movie/collection/items/recommendations/
/movie/recommendations/


### 1.3
**Задача высокого уровня** Сформируйте объект класса dict, где ключ - это URL, а значение - это массив объектов python `datetime.datetime`: когда происходили запросы к этому URL.

*Результат выполнения*
<pre>
{
    '/movie/is_personalizable/': [datetime.datetime(2019, 2, 5, 21, 36, 7)],
    '/logger/content/time/': [datetime.datetime(2019, 2, 5, 21, 36, 7)],
    '/movie/collection/items/recommendations/': [datetime.datetime(2019, 2, 5, 21, 36, 7)],
    '/movie/recommendations/': [datetime.datetime(2019, 2, 5, 21, 36, 7)]
}
</pre>

In [3]:
import datetime
file_path = './data/uwsgi.log'

access_journal = dict()
date2 = []
ip_counter = dict()

with open(file_path, 'r', encoding='utf8') as f:
    for row in f:
        parted_row = row.split(' ')
        url_list = parted_row[6].split('?')[0]        
        date1 = parted_row[3].split('/')
        date2 = date1[2].split(':')
        if date1[1] == 'JAN':
            date1[1] = 1
        elif date1[1] == 'FEB':
            date1[1] = 2
            
# ... много месяцев, которые какими-то буквами обозначаются в логах.

        elif (date1[1] != 'DEC'):
            date1[1] = 12
        date1[0] = int(date1[0][1:])
        
        access_journal[url_list] = datetime.datetime(int(date2[0]), date1[1], date1[0], int(date2[1]), int(date2[2]), int(date2[3]))
print(access_journal)

{'/movie/is_personalizable/': datetime.datetime(2019, 12, 5, 21, 36, 7), '/logger/content/time/': datetime.datetime(2019, 12, 5, 21, 36, 7), '/movie/collection/items/recommendations/': datetime.datetime(2019, 12, 5, 21, 36, 7), '/movie/recommendations/': datetime.datetime(2019, 12, 5, 21, 36, 7)}


## 2. Чтение csv файлов: модуль python csv

### 2.1
**Простая задача** Сформируйте словарь, в котором ключ - название столбца, а значение - массив записей в этом столбце.

*Результат выполнения*
<pre>
{
    'Код': ['HYDRA-535', 'HYDRA-534', 'HYDRA-532', 'HYDRA-531', 'HYDRA-530', 'HYDRA-527', 'HYDRA-524', 'HYDRA-523', 'HYDRA-520', 'HYDRA-519', 'HYDRA-518', 'HYDRA-517', 'HYDRA-514', 'HYDRA-513', 'HYDRA-511', 'HYDRA-510', 'HYDRA-509', 'HYDRA-507', 'HYDRA-506'],
    'Тема': ['Пробрасывать пользовательское распределение paid_types в ехидну', 'Гибридный рекомендатель с multi-channel feedback', 'Джоба в дженкинсе для расчёта динамики РВП', 'Интеграция Hydra с Gamora', 'Тестируем интеграцию с Jira', 'Поправить функцию _get_ui_rec_matrix', 'Оптимизировать матрицу ItemFactors', 'Сортировка ЦПБ', 'Закостылить параметр top', "Сделать 'stable' конфигом по умолчанию в Гидре", 'Неудобно тестировать запись в redis', 'Улучшить рекомендации (первая итерация)', 'Добавить логирование в скрипты hydra/bin', 'Поправить storage_backend', 'Перемешивание рекомендаций для старых пользователей', 'Поправить скрипты bpr и и оценщика', 'Динамические персональные рекомендации', 'Навести порядок в prepare_data_for_hydra', 'Техдолг по логике /collection/recommendations/'],
    'Компонент': ['echidna', 'hydra', 'hydramatrices', 'hydramagrices', 'hydra', 'hydra', 'hydra', 'hydra', 'hydra', 'hydra', 'hydramatrices', 'hydra', 'hydramagrices', 'hydramatrices', 'hydra', 'hydra_utils', 'hydra', 'hydramagrices', 'hydra'],
    'Затрачено в часах': ['1', '3', '2', '4', '2', '10', '2', '5', '2', '2', '1', '7', '5', '2', '5', '16', '10', '3', '24']
}
</pre>

In [4]:
import csv
columns_dict = dict()
file_path = './data/task.csv'

# решение
with open(file_path, newline='', encoding='utf8') as csvfile:
    reader = csv.DictReader(csvfile)
    for col_name in reader.fieldnames:
        columns_dict[col_name] = []
    for row in reader:
        for column in row:
            columns_dict[column].append(row[column])
print(columns_dict)

{'Код': ['HYDRA-535', 'HYDRA-534', 'HYDRA-532', 'HYDRA-531', 'HYDRA-530', 'HYDRA-527', 'HYDRA-524', 'HYDRA-523', 'HYDRA-520', 'HYDRA-519', 'HYDRA-518', 'HYDRA-517', 'HYDRA-514', 'HYDRA-513', 'HYDRA-511', 'HYDRA-510', 'HYDRA-509', 'HYDRA-507', 'HYDRA-506'], 'Тема': ['Пробрасывать пользовательское распределение paid_types в ехидну', 'Гибридный рекомендатель с multi-channel feedback', 'Джоба в дженкинсе для расчёта динамики РВП', 'Интеграция Hydra с Gamora', 'Тестируем интеграцию с Jira', 'Поправить функцию _get_ui_rec_matrix', 'Оптимизировать матрицу ItemFactors', 'Сортировка ЦПБ', 'Закостылить параметр top', "Сделать 'stable' конфигом по умолчанию в Гидре", 'Неудобно тестировать запись в redis', 'Улучшить рекомендации (первая итерация)', 'Добавить логирование в скрипты hydra/bin', 'Поправить storage_backend', 'Перемешивание рекомендаций для старых пользователей', 'Поправить скрипты bpr и и оценщика', 'Динамические персональные рекомендации', 'Навести порядок в prepare_data_for_hydra', '

### 2.2
**Задача среднего уровня** Посчитайте статистику по колонке "Затрачено в часах" (это человеко-часы, затраченные на задачу) – максимальное значение, минимальное значение, среднее значение. Воспользуйтесь словарём `columns_dict` из задания 2.1

*Результат выполнения*
<pre>
min_val=1, max_val=24, mean_val=5.578947368421052

</pre>

In [5]:
min_val, max_val, mean_val = None, None, None


# решение
min_val = min(columns_dict['Затрачено в часах'])
max_val = max(columns_dict['Затрачено в часах'])
int_column = [int(i) for i in columns_dict['Затрачено в часах']]
mean_val = sum(int_column)/len(columns_dict['Затрачено в часах'])
print("min_val={}, max_val={}, mean_val={}".format(min_val, max_val, mean_val))

min_val=1, max_val=7, mean_val=5.578947368421052


### 2.3
**Задача высокого уровня** Сколько человеко-часов было потрачено на задачи, в которых встречается слово "рекомендации" в любых формах, а сколько на остальные задачи (в процентах)?

Подсказка - проверка на наличие нужного слова в тексте `txt` выглядит так:

In [3]:
print('рек' in 'Я делаю рекомендательную систему', 'рек' in 'кек')

True False


*Результат выполнения*
<pre>
{'recs': 2, 'non_recs': 17}
</pre>

In [8]:
file_path = './data/task.csv'
task_recs_counter = {'recs': 1, 'non_recs': 2}
recs_counter = 0
nonrecs_counter = 0

with open(file_path, newline='', encoding='utf8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if 'рекомендац' in row['Тема']:
            recs_counter += int(row['Затрачено в часах'])
        else:
            nonrecs_counter += int(row['Затрачено в часах'])
    task_recs_counter['non_recs'] = 100 * nonrecs_counter / (nonrecs_counter + recs_counter)
    task_recs_counter['recs'] = 100 * recs_counter / (recs_counter + nonrecs_counter)
print(task_recs_counter)

{'recs': 20.754716981132077, 'non_recs': 79.24528301886792}


## 3. Чтение csv файлов: модуль pandas

### 3.1

**Простое задание** добавьте столбец `num_tasks`, где отражено количество задач, выполненное внутри каждого компонента. Колонка `mean_val` со средним значением поля 'Затрачено в часах' уже добавлена в качестве примера.

Чтобы выполнить задание, поправьте словарь `aggregation_json`

In [11]:
import numpy as np
import pandas as pd

file_path = './data/task.csv'

df = pd.read_csv(file_path)

## -- ВАШ КОД ТУТ --
aggregation_json = {
 'Затрачено в часах': {
  'mean_val': np.mean,
  'size_val': np.size}
}

df.groupby(by=['Компонент'])[['Затрачено в часах']].aggregate(aggregation_json)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\groupby\groupby.py:4658: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


,mean_val,size_val
Компонент,,
echidna,1.000000,1
hydra,6.545455,11
hydra_utils,16.000000,1
hydramagrices,4.000000,3
hydramatrices,1.666667,3


### 3.2
**Задание среднего уровня** Добавьте (в виде `list`) для каждого компонента набор задач, который был в этом компоненте выполнен (идентификаторы содержатся в поле "Код"). Для этого замените `.aggregate()` на `.apply(list)`

Пример:

In [14]:
import pandas as pd

print('Исходный DataFrame:')
a = pd.DataFrame({'key':['a', 'a'], 'val':[1, 2]})
print(a)
print('Применяем apply:')
a.groupby('key')['val'].apply(list)

Исходный DataFrame:
  key  val
0   a    1
1   a    2
Применяем apply:


key
a    [1, 2]
Name: val, dtype: object

Результат выполнения

<pre>
Компонент
echidna                                                [HYDRA-535]
hydra            [HYDRA-534, HYDRA-530, HYDRA-527, HYDRA-524, H...
hydra_utils                                            [HYDRA-510]
hydramagrices                    [HYDRA-531, HYDRA-514, HYDRA-507]
hydramatrices                    [HYDRA-532, HYDRA-518, HYDRA-513]
</pre>

In [12]:
# решение
codes_list_df = df.groupby('Компонент')['Код'].apply(list).head()

codes_list_df.head()

Компонент
echidna                                                [HYDRA-535]
hydra            [HYDRA-534, HYDRA-530, HYDRA-527, HYDRA-524, H...
hydra_utils                                            [HYDRA-510]
hydramagrices                    [HYDRA-531, HYDRA-514, HYDRA-507]
hydramatrices                    [HYDRA-532, HYDRA-518, HYDRA-513]
Name: Код, dtype: object

### 3.3
**Задание высокого уровня** Составьте DataFrame с двумя колонками: среднее время на задачу и список задач `codes_list_df` внутри каждого "Компонента". Подсказка: подготовьте два датафрейма и используйте [функцию merge](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html)

In [13]:
# решение

## -- ВАШ КОД ТУТ --
aggregation_json = {
 'Затрачено в часах': {'mean_val': np.mean}
}

df.groupby('Компонент')['Затрачено в часах'].aggregate(aggregation_json).merge(codes_list_df.to_frame(), left_on = 'Компонент', right_on = 'Компонент')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\groupby\groupby.py:3558: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  results[name] = obj.aggregate(func)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:543: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


,"(Затрачено в часах, mean_val)",Код
Компонент,,
echidna,1.000000,[HYDRA-535]
hydra,6.545455,"[HYDRA-534, HYDRA-530, HYDRA-527, HYDRA-524, H..."
hydra_utils,16.000000,[HYDRA-510]
hydramagrices,4.000000,"[HYDRA-531, HYDRA-514, HYDRA-507]"
hydramatrices,1.666667,"[HYDRA-532, HYDRA-518, HYDRA-513]"


## 4.  Работа с данными формата HTML

### 4.1

**Задание простого уровня**. Мы вывели весь список курсов на странице. Измените вывод и добавьте в каждому курсу ссылку http на этот курс на сайте skillbox (чтобы решить задачу, исследуйте объект course_dict).

In [14]:
# собираем вспомогательные данные
import requests
from lxml import html

page = requests.get('https://skillbox.ru/courses/code').content

html_tree = html.fromstring(page)

items = html_tree.xpath("//div[contains(@class, '{}')]".format('header-menu__label'))

items_list = html_tree.xpath("//a[@class='{}'] ".format('header-menu__link'))

*Результат выполнения*
<pre>
Курс: Веб-дизайн с нуля до PRO  link:  https://skillbox.ru/webdesign/
Курс: Рекламная графика  link:  https://skillbox.ru/cpeople/
Курс: UX-дизайн  link:  https://skillbox.ru/aic/
</pre>

In [15]:
for course in items_list:
    course_dict = dict(course.items())
    print(course_dict['title'] + '  link: ' + course_dict['href'])

Курс: Веб-дизайн с нуля до PRO  link: https://skillbox.ru/webdesign/
Курс: Рекламная графика  link: https://skillbox.ru/cpeople/
Курс: UX-дизайн  link: https://skillbox.ru/aic/
Курс: Анимация интерфейсов  link: https://skillbox.ru/motion/
Курс: Дизайн мобильных приложений  link: https://skillbox.ru/rmr/
Курс: Промо-сайты от Red Collar  link: https://skillbox.ru/redcollar/
Курс: Дизайн интерьеров с нуля  link: https://skillbox.ru/interiordesign/
Курс: Графический дизайнер с нуля до PRO  link: https://skillbox.ru/graphic-design/
Курс: Скетчинг для дизайнеров  link: https://skillbox.ru/sketching/
Курс: Photoshop c 0 до PRO  link: https://skillbox.ru/photoshoppro/
Курс: Sound-design  link: https://skillbox.ru/sound-design/
Курс: Cinema4D  link: https://skillbox.ru/cinema4d/
Курс: Sketch  link: https://skillbox.ru/sketchapp/
Курс: Figma  link: https://skillbox.ru/figma/
Курс: Веб-разработчик c 0 до PRO  link: https://course.skillbox.ru/profession-webdev
Курс: Мобильный разработчик с нуля  l

### 4.2
**Задание среднего уровня** Распечатайте список курсов только из раздела "Программирование".

In [16]:
import requests
from lxml import html

page = requests.get('https://skillbox.ru/courses/code').content

html_tree = html.fromstring(page)

# формируем список разделов
items_menu_group = html_tree.xpath("//div[contains(@class, '{}')]".format('header-menu__group'))

## -- ВАШ КОД ТУТ --

# находим нужный индекс среди доступных разделов
relevant_index = 1
# выбираем нужный раздел сайта из 4-х доступных
relevant_group = items_menu_group[relevant_index]
# Код, чтобы выбрать нужный элемент html-верстки. Оператор "." обязателен

items_list = relevant_group.xpath(".//a")

for course in items_list:
    course_dict = dict(course.items())
    print(course_dict['title'])

Курс: Веб-разработчик c 0 до PRO
Курс: Мобильный разработчик с нуля
Курс: Java-разработчик
Курс: PHP-разработчик с 0 до PRO
Курс: Python-разработчик с нуля
Курс: Frontend-разработчик
Курс: Мобильный разработчик PRO
Курс: Программист 1С-Битрикс
Курс: С#-разработчик с нуля
Курс: Аналитик данных на Python
Курс: Data Scientist с 0 до PRO
Курс: Разработчик игр с 0 до PRO


### 4.3
**Задание высокого уровня**. Напишите код, который переходит по ссылкам на страницы курсов по программированию из предыдущего задания и распечатывает описание курса.

Подсказка - для описания используйте т.н. мета-теги: `course_html_tree.xpath('/html/head/meta[@name="description"]/@content')`

In [17]:
import requests

links = []
descr = []

for course in items_list:
    course_dict = dict(course.items())
    
    page = requests.get(course_dict['href']).content
    course_html_tree = html.fromstring(page)
    items = course_html_tree.xpath(('/html/head/meta[@name="description"]/@content').format('header-menu__link'))

    if items == []:
        descr.append('Нет описания')
    else:
        descr.append(items[0])
    links.append(course_dict['href'])

result_df = pd.DataFrame({
    'link': links,
    'descr': descr
})

result_df.head()

,link,descr
0,https://course.skillbox.ru/profession-webdev,Поможет программа профессиональной подготовки ...
1,https://skillbox.ru/mobile/,Мы обучаем востребованным интернет-профессиям ...
2,https://skillbox.ru/java/,Оставь заявку на курс «Java-разработчик» от Sk...
3,https://skillbox.ru/php/,Оставьте заявку на курс «PHP-разработчик с нул...
4,https://skillbox.ru/python/,Оставьте заявку на курс «Python-разработчик с ...


## 5. Работа с данными формата XML

### 5.1

**Задание начального уровня** Распечатайте все фильмы (а не только драмы) в формате: `название фильма: жанр`

In [2]:
from xml.etree import ElementTree

file_path = './data/xml_content_description.xml'

with open(file_path) as f:
    doc = ElementTree.parse(file_path)
    content_titles = doc.getroot()
    for movie in content_titles.findall('./Content/content_title'):
        if movie.find('./genre') is not None:
            film_genre = movie.find('./genre').text
        else:
            film_genre = 'жанр не указан'
        film_name = movie.find('./title').text
        print(film_name + ': ' + film_genre)

The Shawshank Redemption: drama
The Dark Knight: drama
"Back 2 the Future": жанр не указан


### 5.2
**Задание среднего уровня** Посчитайте среднюю длительность контента в файле. xml_content_description

In [20]:
import math
from xml.etree import ElementTree

file_path = './data/xml_content_description.xml'
duration = []
num_content = 0

with open(file_path) as f:
    doc = ElementTree.parse(file_path)
    content_titles = doc.getroot()
    for movie in content_titles.findall('./Content/content_title'):
        if movie.find('./duration') is not None:
            num_content += 1
            duration.append(int(movie.find('./duration').text))
            print(movie.find('./duration').text)

print("avg_duration={}".format(sum(duration)/num_content))

8520
9120
avg_duration=8820.0


### 5.3
**Задание высокого уровня** Если речь идёт о продакшн-системах машинного обучения, то данные могут приходить в форматах, мало пригодных для использования - например XML. В таких случаях приходится приводить данные к превычному табличному формату - такой кейс мы решим в рамках домашней работы.

Нужно определить, сколько полей будет в выходном файле и как залить в него данные

Алгоритм
* сформируйте массив словарей `content_list`, где каждый элемент массива - это словарь вида `{tag1: value1,...,tag_n:value_n}`
* каждый результат findall можно итеративно обойти, распечатав тэг и его значение в виде
<pre>
        for i in movie:
            print(i.text, i.tag)
</pre>
* из полученного словаря сформируйте DataFrame с тремя колонками: `movie_id | tag | value` (movie_id соответствует номеру словаря среди всех словарей `content_list`)
* сохраните Dataframe c помощью функции `.to_csv()`

In [27]:
# имя входного файла XML
input_file = './data/xml_content_description.xml'
# имя выходного файла CSV
output_file = 'csv_content_description.csv'

content_list = []
content_dataframe = pd.DataFrame(columns = ['movie_id', 'tag', 'value'])

# читаем входной файл
with open(file_path) as f:
    doc = ElementTree.parse(file_path)
    content_titles = doc.getroot()
    num = 0
    for i in doc.iter():
        content_list.append({'tag' + str(num):i.tag, 'text' + str(num) : i.text})
        print(i.text, i.tag)
        num += 1
    num = 0
    for i in content_list:
        #print(content_dataframe.append([num, i['tag' + str(num)], i['text' + str(num)]]))
       # print(i['tag' + str(num)] + '     ' + i['text' + str(num)])
        content_dataframe.loc[num] = [num, i['tag' + str(num)], i['text' + str(num)]]
        num += 1

# пишем выходной файл
content_dataframe.to_csv('movies.csv')


 collection

     Content

         content_title
1181251680 content_id
drama genre
The Shawshank Redemption title
040000008200E000 iid
1181572063 releaseTime
None state
None location
8520 duration
Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency. descr

         content_title
1234360800 content_id
The Dark Knight title
drama genre
9120 duration
The Dark Knight must accept one of the greatest psychological and physical tests of his ability to fight injustice. descr
None location
604f4792-eb89-478b-a14f-dd34d3cc6c21-1234360800 iid
dismissed state

    Content

	 content_title
"Back 2 the Future" title
1985 year
Marty McFly descr
